# Python Polars - Fastest Data Library on CPU and GPU

## Quickstart
This section will introduce you to Lazy Frames and some of their basic operations.

### Create LazyFrame
In the following cell, we will create a new Lazy Frame with 2 columns - name and age, along with 2 entries.

In [1]:
import polars as pl

pl.LazyFrame({"name": ["Batman","Gandalf"], "age": [35, 55000]}).collect()

name,age
str,i64
"""Batman""",35
"""Gandalf""",55000


### Polars Data Structures

Polars deals with 2 main data structures - Lazy Frames and Data Frames

In [2]:
print(type(pl.LazyFrame({"name": ["Batman","Gandalf"], "age": [35, 55000]}).collect()))
print(type(pl.LazyFrame({"name": ["Batman","Gandalf"], "age": [35, 55000]})))

<class 'polars.dataframe.frame.DataFrame'>
<class 'polars.lazyframe.frame.LazyFrame'>


### Filtering

We can use Lazy Frames to filter data, selecting only the rows where the "age" column value is greater than 100

In [3]:
pl.LazyFrame({"name": ["Batman","Gandalf"], "age": [35, 55000]}).filter(pl.col("age") > 100).collect()

name,age
str,i64
"""Gandalf""",55000


### Selecting

We can also use Lazy Frames to select data summary, such as the sum of all ages.

In [4]:
pl.LazyFrame({"name": ["Batman","Gandalf"], "age": [35, 55000]}).select(pl.col("age").sum()).collect()

age
i64
55035


### GPU Processing

If we are dealing with large datasets or complex operations, the new Poalrs GPU engine can be used to accelarate processes.

#### Install GPU Engine

The following command will install the engine

In [ ]:
!pip install polars[gpu] --extra-index-url=https://pypi.nvidia.com

#### Use GPU Engine
The following command will run on GPU instead of CPU

In [5]:
pl.LazyFrame({"name": ["Batman","Gandalf"], "age": [35, 55000]}).filter(pl.col("age") > 100).collect(engine="gpu")

name,age
str,i64
"""Gandalf""",55000


## Workflow

In this section, we will analyze the <a href="https://www.kaggle.com/datasets/conorsully1/simulated-transactions">Simulated Transactions Dataset</a> from Kaggle.

### Reading CSV
Here's how we can read a CSV file

In [6]:
import polars as pl

data = pl.scan_csv("transactions.csv")
data.head(3).collect(engine="gpu")

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,str,str,str,str,i64,i64,i64,str,f64
"""CI6XLYUMQK""","""2015-05-01""",null,"""T8I9ZB5A6X90UG8""","""2015-09-11""",2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""","""2015-05-01""",null,"""TZ4JSLS7SC7FO9H""","""2017-02-08""",2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""","""2015-05-01""",null,"""TTUKRDDJ6B6F42H""","""2015-08-01""",2015,8,1,"""Housing""",383.8


### Compress CSV to Parquet
To ensure we are dealing with optimized file formats, we will compress our CSV file into a Parquet file, shrinking 22GB into 4GB.

In [7]:
data = pl.scan_csv("transactions.csv")
data.sink_parquet("transactions.parquet")

### Scan Parquet
Then, we can read our new efficient format file with the following commands, and display its column names.

In [21]:
data = pl.scan_parquet("transactions.parquet")
data.collect_schema()

Schema([('CUST_ID', String),
        ('START_DATE', Date),
        ('END_DATE', Date),
        ('TRANS_ID', String),
        ('DATE', Date),
        ('YEAR', Int64),
        ('MONTH', Int64),
        ('DAY', Int64),
        ('EXP_TYPE', String),
        ('AMOUNT', Float64)])

### Select Statement
Then, we can use a select statement to find the number of unique customers.

In [3]:
data.select(pl.col("CUST_ID").n_unique()).collect(engine="gpu")

CUST_ID
i32
75000


### Filter Statement

Then we can use a set of filters to find the sum of all Housing transactions for year 2020, as well as display the result as an integera rather than a float.

In [4]:
data.filter(
    (pl.col("YEAR") == 2020) &
    (pl.col("EXP_TYPE") == "Housing")
).select(
    pl.sum("AMOUNT").cast(pl.Int64)
).collect(engine="gpu")

AMOUNT
i64
369592978


### Group By Statement

We can also display data highlights by grouping our dataset into expense type categories, and for each category displaying the sum, mean and count of transactions.

In [11]:
data.group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT"),
    pl.mean("AMOUNT").alias("AVERAGE"),
    pl.count("AMOUNT").alias("COUNT")
).sort(
    by="AVERAGE", descending=True
).collect(engine="gpu")

EXP_TYPE,AMOUNT,AVERAGE,COUNT
str,f64,f64,u32
"""Housing""",2.6891e9,1558.747839,1725174
"""Tax""",1.2156e9,412.951411,2943660
"""Education""",9.5917e8,281.19211,3411069
"""Savings""",4.8526e8,223.370883,2172456
"""Bills and Utilities""",1.7967e9,208.2844,8626318
…,…,…,…
"""Fines""",8.0742e6,159.200791,50717
"""Motor/Travel""",3.8798e9,133.380649,29087820
"""Gambling""",4.9470e8,105.020112,4710532


## Data Visualization

### Bar Chart
We can also plot the results of our group by operation in a bar chart.

In [14]:
data.group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT"),
    pl.mean("AMOUNT").alias("AVERAGE"),
    pl.count("AMOUNT").alias("COUNT")
).sort(
    by="AVERAGE", descending=True
).collect(engine="gpu").plot.bar(x="EXP_TYPE", y="AVERAGE")

alt.Chart(...)

### Scatter Plot
Additionaly, we can plot a complex scatter plot visualization with millions of data nodes (please note, I'm only plotting 10,000 to keep the size of the notebook below 25MB).

In [23]:
import altair as alt
alt.data_transformers.enable("vegafusion")

data.head(10000).collect(engine="gpu").plot.scatter(
    x="AMOUNT", y="DATE", color="EXP_TYPE", tooltip=[
        "TRANS_ID", "CUST_ID", "AMOUNT", "DATE", "EXP_TYPE"
    ]
).properties(
    width=750, title="Expense Amounts and Types Over Time"
).mark_circle(
    size=25, strokeWidth=1
)

alt.Chart(...)

## Error Solving
If you run into any errors or unexpected results, please try downloading this version of the parquet file:

In [19]:
!wget https://storage.googleapis.com/rapidsai/polars-demo/transactions.parquet -O transactions.parquet

--2024-10-23 08:03:13--  https://storage.googleapis.com/rapidsai/polars-demo/transactions.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.217.91, 142.251.211.251, 142.250.217.123, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.217.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4274457161 (4.0G) [application/octet-stream]
Saving to: ‘transactions.parquet’

transactions.parque 100%[===================>]   3.98G  66.2MB/s    in 58s     

2024-10-23 08:04:11 (69.7 MB/s) - ‘transactions.parquet’ saved [4274457161/4274457161]



## Thanks for Reading!